In [1]:
from kaggle_secrets import UserSecretsClient
secret_label = "Token"
token = UserSecretsClient().get_secret(secret_label)

! git clone https://{token}@github.com/CarloSgaravatti/Recsys_Challenge_2023.git

Cloning into 'Recsys_Challenge_2023'...
remote: Enumerating objects: 1404, done.
remote: Counting objects: 100% (253/253), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 1404 (delta 184), reused 193 (delta 143), pack-reused 1151
Receiving objects: 100% (1404/1404), 78.85 MiB | 29.48 MiB/s, done.
Resolving deltas: 100% (1014/1014), done.


In [2]:
! pip install PyGithub requests

  Obtaining dependency information for PyGithub from https://files.pythonhosted.org/packages/be/04/810d131be173cba445d3658a45512b2b2b3d0960d52c4a300d6ec5e00f52/PyGithub-2.1.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 20.9 MB/s eta 0:00:00


In [3]:
! cd /kaggle/working/Recsys_Challenge_2023 && python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorization_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorization_Cython_Epoch.c:1109:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorization_Cython_Epoch.c: In function ‘__pyx_pf_32MatrixFactorization

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
import random
import tqdm
import gc
import time
import os
import shutil
import optuna
import scipy.sparse as sps
from concurrent.futures import ThreadPoolExecutor

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv
/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv
/kaggle/input/training-tuned-models/__results__.html
/kaggle/input/training-tuned-models/__notebook__.ipynb
/kaggle/input/training-tuned-models/ials_xgboost_train_validation.zip
/kaggle/input/training-tuned-models/ials_train_validation.zip
/kaggle/input/training-tuned-models/__output__.json
/kaggle/input/training-tuned-models/ials_all.zip
/kaggle/input/training-tuned-models/custom.css
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/run_all_algorithms.py
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/cython_example_FunkSVD.py
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/LICENSE
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/.gitignore
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/rp3-tuning.ipynb
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/README.md
/kaggle/input/trai

In [5]:
%cd /kaggle/working/Recsys_Challenge_2023

/kaggle/working/Recsys_Challenge_2023


In [6]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender, PureSVDItemRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.Neural.MultVAERecommender import MultVAERecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.SLIM.SLIM_BPR_Python import SLIM_BPR_Python
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender
from Recommenders.BaseRecommender import BaseRecommender

from Recommenders.Hybrid.LinearCombinationRecommender import LinearCombinationRecommender
from Data_manager.CrossValidationSplitter import CrossValidationSplitter

np.int = int
np.random.seed(42)
np.bool = bool
np.float = float

In [7]:
from github import Github
from github import Auth

auth = Auth.Token(token)
g = Github(auth=auth)

repo = None
for r in g.get_user().get_repos():
    if r.name == 'Recsys_Challenge_2023':
        repo = r
        print('Repository found')

Repository found


In [8]:
def upload_file(filepath_kaggle, filepath_github, commit_message):
    try:
        contents = repo.get_contents(filepath_github)
        with open(filepath_kaggle, "rb") as file:
            repo.update_file(contents.path, commit_message, file.read(), contents.sha)
    except Exception:
        with open(filepath_kaggle, "rb") as file:
            repo.create_file(filepath_github, commit_message, file.read())

In [9]:
config = {
    'tune_parameters': True,
    'database_path': '/kaggle/working/tuning_hybrid_linear_comb.db',
    'copy_prev_tuning_db': True,
    'copy_prev_best_params': True,
    'save_github': True
}

In [10]:
train_data = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv')
user_ids_test = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv')

In [11]:
URM_all = sps.csr_matrix((train_data.data.values,
                          ((train_data.row - 1).values, (train_data.col - 1).values)),
                        shape=(train_data.row.max(), train_data.col.max()))
URM_all

<13024x22347 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [12]:
URM_train, URM_test_complete = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_validation, URM_test = split_train_in_two_percentage_global_sample(URM_test_complete, train_percentage = 0.8)

In [13]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_knn_item.json', 'r') as params_file:
    params_item_knn = json.load(params_file)
params_item_knn

{'topK': 14,
 'shrink': 25,
 'similarity': 'tversky',
 'tversky_alpha': 0.18163095712123836,
 'tversky_beta': 1.8529812241698465}

In [14]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_RP3.json', 'r') as params_file:
    params_rp3 = json.load(params_file)
params_rp3

{'alpha': 0.26348679053501006, 'beta': 0.17536156577039744, 'topK': 27}

In [15]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_knn_user.json', 'r') as params_file:
    params_user_knn = json.load(params_file)
params_user_knn

{'topK': 498,
 'shrink': 0,
 'similarity': 'asymmetric',
 'asymmetric_alpha': 0.39792525383867455}

In [16]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_alpha.json', 'r') as params_file:
    params_p3alpha = json.load(params_file)
params_p3alpha

{'normalize_similarity': 'True',
 'implicit': 'True',
 'alpha': 0.3014708045867471,
 'topK': 36}

In [17]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_hybrid_merging.json', 'r') as params_file:
    params_merging = json.load(params_file)
params_merging

{'alpha': 0.3375591188839621, 'topK': 86}

In [18]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/pure_svd_params.json', 'r') as params_file:
    params_pure_svd = json.load(params_file)
params_pure_svd

{'factors': 30}

In [19]:
# with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_puresvd_item.json', 'r') as params_file:
#     params_pure_svd_item = json.load(params_file)
params_pure_svd_item = {"topK": 43, "num_factors": 171}

In [20]:
shutil.copyfile(
    '/kaggle/input/training-tuned-models/ials_train_validation.zip', 
    '/kaggle/working/ials_train_validation.zip', 
)
shutil.copyfile(
    '/kaggle/input/training-tuned-models/ials_train_validation.zip', 
    '/kaggle/working/ials_all.zip', 
)

'/kaggle/working/ials_all.zip'

In [21]:
recommender_item = ItemKNNCFRecommender(URM_train)
recommender_item.fit(**params_item_knn)

recommender_user = UserKNNCFRecommender(URM_train)
recommender_user.fit(**params_user_knn)

recommender_rp3 = RP3betaRecommender(URM_train)
recommender_rp3.fit(**params_rp3)

recommender_p3a = P3alphaRecommender(URM_train)
recommender_p3a.fit(**params_p3alpha)

recommender_pure_svd = PureSVDRecommender(URM_train)
recommender_pure_svd.fit(num_factors=params_pure_svd['factors'])

recommender_pure_svd_item = PureSVDItemRecommender(URM_train)
recommender_pure_svd_item.fit(**params_pure_svd_item)

recommender_nmf = NMFRecommender(URM_train)
recommender_nmf.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'nmf_train_validation.zip')

recommender_slim_bpr = SLIM_BPR_Python(URM_train)
recommender_slim_bpr.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'slim_bpr_train_validation.zip')

recommender_ease_r = EASE_R_Recommender(URM_train)
recommender_ease_r.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'ease_r_train_validation.zip')

recommender_ials = IALSRecommender(URM_train)
recommender_ials.load_model('/kaggle/working/', 'ials_train_validation.zip')

hybrid_similarity = ItemKNNSimilarityHybridRecommender(URM_train, recommender_item.W_sparse, recommender_rp3.W_sparse)
hybrid_similarity.fit(**params_merging)

ItemKNNCFRecommender: URM Detected 627 ( 4.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 247 ( 1.1%) items with no interactions.
Similarity column 22347 (100.0%), 4565.05 column/sec. Elapsed time 4.90 sec
UserKNNCFRecommender: URM Detected 627 ( 4.8%) users with no interactions.
UserKNNCFRecommender: URM Detected 247 ( 1.1%) items with no interactions.
Similarity column 13024 (100.0%), 4603.53 column/sec. Elapsed time 2.83 sec
RP3betaRecommender: URM Detected 627 ( 4.8%) users with no interactions.
RP3betaRecommender: URM Detected 247 ( 1.1%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 2664.23 column/sec. Elapsed time 8.39 sec
P3alphaRecommender: URM Detected 627 ( 4.8%) users with no interactions.
P3alphaRecommender: URM Detected 247 ( 1.1%) items with no interactions.
P3alphaRecommender: Similarity column 22347 (100.0%), 3235.46 column/sec. Elapsed time 6.91 sec
PureSVDRecommender: URM Detected 627 ( 4.8%) users with no inter

In [22]:
weights = [0.4, 0.4, 0.2]
linear_combinator = LinearCombinationRecommender(URM_train, [recommender_item, recommender_rp3, recommender_p3a], weights)
evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])

print('Linear Combinator')
result_df, _ = evaluator.evaluateRecommender(linear_combinator)
display(result_df)
print('Merging Models')
result_df, _ = evaluator.evaluateRecommender(hybrid_similarity)
display(result_df)
print('Item KNN')
result_df, _ = evaluator.evaluateRecommender(recommender_item)
display(result_df)
print('RP3 beta')
result_df, _ = evaluator.evaluateRecommender(recommender_rp3)
display(result_df)
print('P3 alpha')
result_df, _ = evaluator.evaluateRecommender(recommender_p3a)
display(result_df)
print('User KNN')
result_df, _ = evaluator.evaluateRecommender(recommender_user)
display(result_df)
print('NMF')
result_df, _ = evaluator.evaluateRecommender(recommender_nmf)
display(result_df)
print('Pure SVD')
result_df, _ = evaluator.evaluateRecommender(recommender_pure_svd)
display(result_df)
print('Item Pure SVD')
result_df, _ = evaluator.evaluateRecommender(recommender_pure_svd_item)
display(result_df)
print('SLIM BPR')
result_df, _ = evaluator.evaluateRecommender(recommender_slim_bpr)
display(result_df)
print('EASE R')
result_df, _ = evaluator.evaluateRecommender(recommender_ease_r)
display(result_df)
print('IALS')
result_df, _ = evaluator.evaluateRecommender(recommender_ials)
display(result_df)

Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
Linear Combinator
EvaluatorHoldout: Processed 10456 (100.0%) in 16.93 sec. Users per second: 618


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.096739,0.165159,0.138933,0.050207,0.085675,0.27459,0.152722,0.114059,0.518076,0.366351,...,0.802826,0.415924,0.802826,0.08285,10.654673,0.998407,0.239054,0.820795,1.522478,0.308761


Merging Models
EvaluatorHoldout: Processed 10456 (100.0%) in 9.09 sec. Users per second: 1150


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.097102,0.165756,0.139351,0.050173,0.085482,0.27428,0.152762,0.114452,0.520658,0.365828,...,0.802826,0.417998,0.802826,0.087463,10.729146,0.998493,0.252365,0.826532,1.477653,0.310478


Item KNN
EvaluatorHoldout: Processed 10456 (100.0%) in 9.00 sec. Users per second: 1162


...

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.054524,0.094253,0.079452,0.0259,0.044669,0.176272,0.086926,0.064669,0.36754,0.212465,...,0.802826,0.295071,0.802826,0.034099,9.527262,0.996677,0.098389,0.733943,2.03833,0.294086


EASE R
EvaluatorHoldout: Processed 10456 (100.0%) in 14.20 sec. Users per second: 736


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.074857,0.135323,0.116287,0.037511,0.067475,0.233553,0.123573,0.091082,0.459353,0.293884,...,0.802826,0.368781,0.802826,0.019069,8.350538,0.991242,0.055022,0.643293,3.089786,0.267475


IALS
EvaluatorHoldout: Processed 10456 (100.0%) in 14.59 sec. Users per second: 717


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.08224,0.144123,0.122645,0.041501,0.071584,0.246255,0.131701,0.098458,0.479055,0.316769,...,0.802826,0.384598,0.802826,0.029074,9.74689,0.998532,0.08389,0.750862,1.599865,0.289534


In [23]:
weights = [0.5, 0.8, 0.8, 0.1, 0.05, 0.01, 0.005, 0.005, 0.005]
recommenders = [recommender_item, recommender_rp3, recommender_p3a, recommender_ials, recommender_ease_r, recommender_user, recommender_pure_svd_item, recommender_nmf, recommender_slim_bpr]
linear_combinator = LinearCombinationRecommender(URM_train, recommenders, weights)
evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])
result_df, _ = evaluator.evaluateRecommender(linear_combinator)
display(result_df)

Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 10456 (100.0%) in 44.52 sec. Users per second: 235


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.098336,0.170432,0.14415,0.050739,0.087288,0.278079,0.155784,0.116915,0.52764,0.370649,...,0.802826,0.423603,0.802826,0.066581,10.320409,0.997785,0.192112,0.795044,1.78556,0.300156


In [24]:
evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])

def objective(trial):
    weights = [
        trial.suggest_float('item_knn', 1e-2, 1, log=True),
        trial.suggest_float('rp3', 1e-2, 1, log=True),
        trial.suggest_float('p3alpha', 1e-2, 1, log=True),
        trial.suggest_float('ials', 5e-3, 1, log=True),
        trial.suggest_float('ease_r', 1e-3, 1, log=True),
        trial.suggest_float('user_knn', 1e-3, 1, log=True),
        trial.suggest_float('svd_item', 1e-4, 1, log=True),
        trial.suggest_float('nmf', 1e-5, 1, log=True),
        trial.suggest_float('slim_bpr', 1e-5, 1, log=True)
    ]
    
    recommenders = [
        recommender_item, recommender_rp3, recommender_p3a, recommender_ials, recommender_ease_r, 
        recommender_user, recommender_pure_svd_item, recommender_nmf, recommender_slim_bpr
    ]
    
    linear_combinator = LinearCombinationRecommender(URM_train, recommenders, weights)
    evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])
    result_df, _ = evaluator.evaluateRecommender(linear_combinator)

    return result_df.loc[10, 'MAP']

EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions


In [25]:
# def evaluate_linear_model(URM_train, URM_valid, weights):
#     recommender_item = ItemKNNCFRecommender(URM_train)
#     recommender_item.fit(**params_item_knn)

#     recommender_rp3 = RP3betaRecommender(URM_train)
#     recommender_rp3.fit(**params_rp3)
    
#     recommender_p3a = P3alphaRecommender(URM_train)
#     recommender_p3a.fit(**params_p3alpha)
    
#     linear_combinator = LinearCombinationRecommender(URM_train, [recommender_item, recommender_rp3, recommender_p3a], weights)    
#     evaluator = EvaluatorHoldout(URM_valid, cutoff_list=[10])
#     result_df, _ = evaluator.evaluateRecommender(linear_combinator)
#     return result_df.loc[10, 'MAP']

# def objective(trial):
#     w1 = trial.suggest_float('w1', 0.01, 0.99)
#     w2 = trial.suggest_float('w2', 0.01, 0.99)
#     w3 = trial.suggest_float('w3', 0.01, 0.99)
    
#     results = [None] * 10
#     with ThreadPoolExecutor(max_workers=10) as executor:
#         futures = []
#         for URM_train, URM_valid in CrossValidationSplitter(10).split(URM_all):
#             futures.append(
#                 executor.submit(evaluate_linear_model, URM_train, URM_valid, [w1, w2, w3])
#             )
#         for i, future in enumerate(futures):
#             results[i] = future.result()
#     return np.mean(results)

In [26]:
try:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/tuning_results/tuning_hybrid_merging.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass

In [27]:
if config['tune_parameters']:
    study = optuna.create_study(direction='maximize', study_name='hyperparameters_tuning_linear_9models_no_cv', 
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)
    study.optimize(objective, n_trials=100)

[I 2023-12-01 21:05:57,964] A new study created in RDB with name: hyperparameters_tuning_linear_9models_no_cv


Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 10456 (100.0%) in 45.44 sec. Users per second: 230


[I 2023-12-01 21:06:43,712] Trial 0 finished with value: 0.04836037787979314 and parameters: {'item_knn': 0.03572328178995846, 'rp3': 0.02390615690774205, 'p3alpha': 0.6081818561861205, 'ials': 0.5617459182259633, 'ease_r': 0.0028037839986746775, 'user_knn': 0.010765016394012578, 'svd_item': 0.003065813794054327, 'nmf': 0.02326120304160313, 'slim_bpr': 0.001670813746134162}. Best is trial 0 with value: 0.04836037787979314.


Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 10456 (100.0%) in 44.93 sec. Users per second: 233


[I 2023-12-01 21:07:28,900] Trial 1 finished with value: 0.048550334887845256 and parameters: {'item_knn': 0.011422927419032808, 'rp3': 0.6468714333418365, 'p3alpha': 0.09019859693246099, 'ials': 0.7968942755605112, 'ease_r': 0.08521752402240347, 'user_knn': 0.00251313736056368, 'svd_item': 0.007170132846352929, 'nmf': 0.0010787764557911843, 'slim_bpr': 6.953173797203158e-05}. Best is trial 1 with value: 0.048550334887845256.


...

[I 2023-12-01 22:19:52,139] Trial 97 finished with value: 0.050717492652506566 and parameters: {'item_knn': 0.14817703804382895, 'rp3': 0.5684234754069224, 'p3alpha': 0.33236673107338727, 'ials': 0.0890167903938447, 'ease_r': 0.008037095717725672, 'user_knn': 0.0029640051712122376, 'svd_item': 0.0015258616744238888, 'nmf': 0.00042849309113878435, 'slim_bpr': 0.0006442411578370319}. Best is trial 86 with value: 0.050937632831761136.


Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 10456 (100.0%) in 44.49 sec. Users per second: 235


[I 2023-12-01 22:20:36,957] Trial 98 finished with value: 0.05091222021836074 and parameters: {'item_knn': 0.12203514542071603, 'rp3': 0.421623790326846, 'p3alpha': 0.2399253833635816, 'ials': 0.1255440554289545, 'ease_r': 0.01013095362853584, 'user_knn': 0.0017054512835934503, 'svd_item': 0.0012652638616248847, 'nmf': 0.0008451908219133294, 'slim_bpr': 0.00039700080278407554}. Best is trial 86 with value: 0.050937632831761136.


Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 10456 (100.0%) in 44.30 sec. Users per second: 236


[I 2023-12-01 22:21:21,584] Trial 99 finished with value: 0.05092542749055816 and parameters: {'item_knn': 0.12364252503955349, 'rp3': 0.4716041632695916, 'p3alpha': 0.2620858555741696, 'ials': 0.10916993785659324, 'ease_r': 0.010839268969788743, 'user_knn': 0.003920110313046692, 'svd_item': 0.0007538167342390345, 'nmf': 0.000805177350133551, 'slim_bpr': 0.0007482372077394788}. Best is trial 86 with value: 0.050937632831761136.


In [28]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

In [29]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_param_importances(study)
    fig.show()

In [30]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_slice(study)
    fig.show()

In [31]:
if config['tune_parameters']:
    with open('/kaggle/working/best_params_hybrid_lr_comb_9models.json', 'w') as params_file:
        json.dump(study.best_params, params_file)
        
    if config['save_github']: 
        upload_file(
            '/kaggle/working/best_params_hybrid_lr_comb_9models.json', 
            'tuning_results/best_params_hybrid_lr_comb_9models.json', 
            'Linear combination of 9 models tuning results (from kaggle notebook)'
        )
    
elif config['copy_prev_best_params']:
    shutil.copyfile(
        '/kaggle/input/linear-combination/best_params_hybrid_lr_comb_9models.json', 
        '/kaggle/working/best_params_hybrid_lr_comb_9models.json'
    )

In [32]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'],
        'tuning_results/tuning_hybrid_merging.db', 
        'Linear combination tuning db updated results (from kaggle notebook)'
    )

In [33]:
with open('/kaggle/working/best_params_hybrid_lr_comb_9models.json', 'r') as params_file:
    params = json.load(params_file)
params

{'item_knn': 0.15778573055111927,
 'rp3': 0.4537190593745236,
 'p3alpha': 0.3332076925657023,
 'ials': 0.18873926545901815,
 'ease_r': 0.008958947376370386,
 'user_knn': 0.001454667740908643,
 'svd_item': 0.00041976516416371556,
 'nmf': 0.0011209447523190098,
 'slim_bpr': 0.00031747086938584915}

In [34]:
recommender_item = ItemKNNCFRecommender(URM_all)
recommender_item.fit(**params_item_knn)

recommender_user = UserKNNCFRecommender(URM_all)
recommender_user.fit(**params_user_knn)

recommender_rp3 = RP3betaRecommender(URM_all)
recommender_rp3.fit(**params_rp3)

recommender_p3a = P3alphaRecommender(URM_all)
recommender_p3a.fit(**params_p3alpha)

recommender_pure_svd = PureSVDRecommender(URM_all)
recommender_pure_svd.fit(num_factors=params_pure_svd['factors'])

recommender_pure_svd_item = PureSVDItemRecommender(URM_all)
recommender_pure_svd_item.fit(**params_pure_svd_item)

recommender_nmf = NMFRecommender(URM_all)
recommender_nmf.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'nmf_all.zip')

recommender_slim_bpr = SLIM_BPR_Python(URM_all)
recommender_slim_bpr.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'slim_bpr_all.zip')

recommender_ease_r = EASE_R_Recommender(URM_all)
recommender_ease_r.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'ease_r_all.zip')

recommender_ials = IALSRecommender(URM_all)
recommender_ials.load_model('/kaggle/working/', 'ials_all.zip')

ItemKNNCFRecommender: URM Detected 386 ( 3.0%) users with no interactions.
ItemKNNCFRecommender: URM Detected 125 ( 0.6%) items with no interactions.
Similarity column 22347 (100.0%), 4158.99 column/sec. Elapsed time 5.37 sec
UserKNNCFRecommender: URM Detected 386 ( 3.0%) users with no interactions.
UserKNNCFRecommender: URM Detected 125 ( 0.6%) items with no interactions.
Similarity column 13024 (100.0%), 3909.59 column/sec. Elapsed time 3.33 sec
RP3betaRecommender: URM Detected 386 ( 3.0%) users with no interactions.
RP3betaRecommender: URM Detected 125 ( 0.6%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 2362.70 column/sec. Elapsed time 9.46 sec
P3alphaRecommender: URM Detected 386 ( 3.0%) users with no interactions.
P3alphaRecommender: URM Detected 125 ( 0.6%) items with no interactions.
P3alphaRecommender: Similarity column 22347 (100.0%), 2960.30 column/sec. Elapsed time 7.55 sec
PureSVDRecommender: URM Detected 386 ( 3.0%) users with no inter

In [35]:
weights = [
    params['item_knn'],
    params['rp3'],
    params['p3alpha'],
    params['ease_r'],
    params['ials'],
    params['user_knn'],
    params['svd_item'],
    params['nmf'],
    params['slim_bpr']
]

recommenders = [
    recommender_item, recommender_rp3, recommender_p3a, recommender_ials, recommender_ease_r, 
    recommender_user, recommender_pure_svd_item, recommender_nmf, recommender_slim_bpr
]

In [36]:
linear_combinator = LinearCombinationRecommender(URM_all, recommenders, weights)
recommendations = linear_combinator.recommend(user_ids_test['user_id'].unique() - 1, cutoff=10, return_scores=False)
recommendations = [' '.join([str(item + 1) for item in r]).strip() for r in recommendations]

submission_df = pd.DataFrame({'user_id': user_ids_test['user_id'].unique(), 'item_list': recommendations})
display(submission_df.head())

submission_df.to_csv('/kaggle/working/submission_linear_comb_9models.csv', index=False)

Recommender_Base_Class: URM Detected 386 ( 3.0%) users with no interactions.
Recommender_Base_Class: URM Detected 125 ( 0.6%) items with no interactions.


,user_id,item_list
0,1,36 101 403 506 123 515 694 254 139 592
1,2,1095 47 28 11 50 1522 12 196 102 656
2,3,59 239 259 857 511 1281 584 536 375 414
3,4,28 249 50 139 171 254 314 7 639 129
4,5,1570 77 116 5138 4 148 131 7033 170 95
